In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional
from flask import Flask, request, jsonify,send_from_directory
import json
#import onnxruntime

In [2]:
# demo code 
app = Flask(__name__)

# global variables
global train_logs 
softmax = nn.Softmax(dim=0)

@app.route('/')
def base():
    return "Base Test is OK" # may return a html file

In [3]:
# interface for users to upload dataset
@app.route('/dataset', methods=["POST"])
def getDataSet():
    global inputs
    global labels 
    request_content = request.get_json()    
    #print(type(request_content))
    
    if(isinstance(request_content, str)):
        request_content = json.loads(request_content)
        #print(type(request_content))
    
    inputs = request_content["inputs"]
    labels = request_content["labels"]
    
    inputs = torch.FloatTensor(inputs)
    labels = torch.FloatTensor(labels)    
    
    response = app.response_class(
        response=json.dumps(request_content),
        status=200,
        mimetype='application/json'
    )
        
    return response
    
# if dataset is meta format, like image, json, yaml, ...
#   students may implement the approach to save the file or store data into an NoSQL database

In [4]:
# Make device agnostic code
device = "cpu"
print(f"device is {device}")

class MyModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MyModel, self).__init__()
        hidden_dim = 20
        self.linear_1 = nn.Linear(input_dim, hidden_dim)
        self.linear_2 = nn.Linear(hidden_dim, hidden_dim)
        self.linear_3 = nn.Linear(hidden_dim, hidden_dim)
        self.linear_4 = nn.Linear(hidden_dim, output_dim)
        #self.LogSoftmax = nn.LogSoftmax(dim=0)
        #self.relu = nn.ReLU()

    def forward(self, inputs):
        outputs = self.linear_1(inputs).requires_grad_()      
        outputs = self.linear_2(outputs).requires_grad_()
        outputs = self.linear_3(outputs).requires_grad_()
        outputs = self.linear_4(outputs).requires_grad_()
        #outputs = self.relu(outputs).requires_grad_()
        #outputs = self.LogSoftmax(outputs).requires_grad_()
        return outputs
        
# 4. Create an instance of the model and send it to target device
torch.manual_seed(42)
model = MyModel(7,2).to(device)

print(model.parameters)# Type of parameter object

device is cpu
<bound method Module.parameters of MyModel(
  (linear_1): Linear(in_features=7, out_features=20, bias=True)
  (linear_2): Linear(in_features=20, out_features=20, bias=True)
  (linear_3): Linear(in_features=20, out_features=20, bias=True)
  (linear_4): Linear(in_features=20, out_features=2, bias=True)
)>


In [5]:
def encodeLabel(onehats,dim):
    labelcoding = []
    for code in onehats:
        value = 0
        for idx in range(dim):
            if(code[idx]==1):
                value = idx
        labelcoding.append(value)
    labelcoding = torch.FloatTensor(labelcoding) 
    return labelcoding  

In [6]:
# Create a loss function
criterion = nn.CrossEntropyLoss()  

# Create an optimizer
learning_rate = 0.0001
optimizer = torch.optim.SGD(params=model.parameters(),lr=learning_rate)

@app.route("/train/start", methods=['GET', 'POST'])
def start():
    return "Busy, system is now training" # may return a html file

@app.route("/train/training", methods=['GET', 'POST'])
def trainModel():
    global train_logs
    global inputs
    global labels
    
    train_logs = []    
    num_epochs = 10000 # Train for longer\
    softmax = nn.Softmax(dim=0)
    
    # Put data to target device
    # device = "cuda" if torch.cuda.is_available() else "cpu"# Enable CUDA if possible
    device = "cpu"
    print(f"device is {device}")
    
    model.to(device)
    
    labels=encodeLabel(labels,2)
    inputs, labels = inputs.to(device), labels.to(device)

    for epoch in range(num_epochs):
        predicts = model(inputs)
        loss = criterion(predicts,labels.long() )  # https://www.cnblogs.com/blogwangwang/p/12018897.html
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print out what's happening every 10 epochs
        if epoch % 100 == 0:
            train_logs.append(f"Epoch: {epoch} | Loss: {loss:.5f}")
    
    return "OK"

@app.route("/train/end", methods=['GET', 'POST'])
def end():
    return "Idle, training is finished." # may return a html file



In [7]:
@app.route("/trainedModel", methods=['GET'])
def getTrainedModel():
    x = [1,2,3]
    x = torch.FloatTensor(x).to(device)
    model.eval()
    torch.onnx.export(model,x,"trainedModel.onnx")
    
    try:
        return send_from_directory('' , 'trainedModel.onnx', as_attachment=True)
    except FileNotFoundError:
        abort(404)



In [8]:
@app.route("/log", methods=['GET'])
def getTrainingLog():    
    response = app.response_class(
        response=json.dumps(train_logs),
        status=200,
        mimetype='application/json'
    )
    return response

In [9]:
@app.route("/inference", methods=['POST'])
def inference():
    device = "cpu"
    request_content = request.get_json() 
    if(isinstance(request_content, str)):
        request_content = json.loads(request_content)
    input_data = request_content["input"]
    
    print(input_data)
    
    input_data = torch.FloatTensor(input_data)
    input_data = input_data.to(device)
    result = softmax(model(input_data)).tolist()
    
    response = app.response_class(
        response=json.dumps(result),
        status=200,
        mimetype='application/json'
    ) 
    return response

In [ ]:
# Run up your Web API Server
if __name__ == '__main__':
    app.run(host='0.0.0.0',port=5000)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://172.17.0.21:5000/ (Press CTRL+C to quit)
192.168.50.109 - - [06/Dec/2023 11:32:31] "POST /inference HTTP/1.1" 200 -


[9.9215716796875, -1.4173673828125, 0.7493851196289062, -6.511450381679389, 6.801526717557252, 0.9236641221374046, 0.0]


192.168.50.109 - - [06/Dec/2023 11:32:33] "POST /inference HTTP/1.1" 200 -


[9.967061511230469, -1.2856862915039062, 0.7206546997070312, -6.5038167938931295, 6.778625954198473, 1.183206106870229, 0.0]


...

[9.993397729492187, -1.3311761230468748, 0.7709329345703124, -6.67175572519084, 6.770992366412214, 1.1297709923664123, 0.0]


192.168.50.109 - - [06/Dec/2023 11:39:12] "POST /inference HTTP/1.1" 200 -


[9.962273107910155, -1.4030021728515625, 0.7350199096679687, -6.603053435114504, 6.786259541984733, 1.3435114503816794, 0.0]


192.168.50.109 - - [06/Dec/2023 11:39:14] "POST /inference HTTP/1.1" 200 -


[9.938331091308592, -1.3407529296874998, 0.7302315063476562, -6.427480916030534, 6.709923664122138, 1.3053435114503817, 0.0]
